In [218]:
# fill the area id column
import os 
import pandas as pd

def fill_area_code(df):

    df['AreaID'].dropna(inplace=True)
    cn_id = df['AreaID'][0]
    print(cn_id)
    df['AreaID'] = cn_id
    
    return df
        

In [219]:
import pandas as pd
import os

def read_and_concatenate(folder_path):
    # Lists to store DataFrames
    gen_dataframes = []
    load_dataframes = []

    # Iterate over all files in the folder
    list_files = [file for file in os.listdir(folder_path) if file!='test.csv']
    for file in list_files:
        if file.endswith('.csv'):
            print('---------------------------------')
            print(file)
            file_path = os.path.join(folder_path, file)
            

            # Read 'gen' files
            if file.startswith('gen'):
                df = pd.read_csv(file_path)
                if file == 'gen_SP_B10.csv':
                    df['AreaID'] = '10YES-REE------0'
                df = fill_area_code(df)
                df['quantity'].fillna(0, inplace=True)
                gen_dataframes.append(df)


            # Read 'load' files
            elif file.startswith('load'):
                df = pd.read_csv(file_path)
                df = fill_area_code(df)
                df['Load'].fillna(0, inplace=True)
                load_dataframes.append(df)

           
    # Concatenate DataFrames vertically
    gen_concatenated = pd.concat(gen_dataframes, axis=0, ignore_index=True)
    load_concatenated = pd.concat(load_dataframes, axis=0, ignore_index=True)

    # Combine 'gen' and 'load' DataFrames
    combined_dataframe = pd.concat([gen_concatenated, load_concatenated], axis=0, ignore_index=True)

    return combined_dataframe

# Provide the path to your folder containing the CSV files
folder_path = '../data/raw_data/'
data = read_and_concatenate(folder_path)

# Optionally, save the combined DataFrame to a new CSV file
#combined_df.to_csv('combined_dataset.csv', index=False)


---------------------------------
gen_DE_B01.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B09.csv


10Y1001A1001A83F
---------------------------------
gen_DE_B10.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B11.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B12.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B15.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B16.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B18.csv
10Y1001A1001A83F
---------------------------------
gen_DE_B19.csv
10Y1001A1001A83F
---------------------------------
gen_DK_B01.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B16.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B18.csv
10Y1001A1001A65H
---------------------------------
gen_DK_B19.csv
10Y1001A1001A65H
---------------------------------
gen_HU_B01.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B09.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B11.csv
10YHU-MAVIR----U
---------------------------------
gen_HU_B12.csv
10YHU-MAVI

In [220]:
data['AreaID'].unique()

array(['10Y1001A1001A83F', '10Y1001A1001A65H', '10YHU-MAVIR----U',
       '10YIT-GRTN-----B', '10YNL----------L', '10YPL-AREA-----S',
       '10YSE-1--------K', '10YES-REE------0', '10Y1001A1001A92E'],
      dtype=object)

In [221]:
data['StartTime'] = pd.to_datetime(data['StartTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['EndTime'] = pd.to_datetime(data['EndTime'].str.replace('\+00:00Z', '', regex=True)).dt.strftime('%Y-%m-%d %H:%M:%S')
data['StartTime'] = pd.to_datetime(data['StartTime'])
data['EndTime'] = pd.to_datetime(data['EndTime'])

In [222]:
data['AreaID'] = data['AreaID'].replace({
    '10YHU-MAVIR----U': 'HU',
    '10YIT-GRTN-----B': 'IT',
    '10YPL-AREA-----S': 'PO',
    '10YES-REE------0': 'SP',
    '10Y1001A1001A92E': 'UK',
    '10Y1001A1001A83F': 'DE',
    '10Y1001A1001A65H': 'DK',
    '10YSE-1--------K': 'SE',
    '10YNL----------L': 'NE'
})

In [223]:
data['AreaID'].unique()

array(['DE', 'DK', 'HU', 'IT', 'NE', 'PO', 'SE', 'SP', 'UK'], dtype=object)

In [224]:
data.fillna(0, inplace=True)
data['gen/load']='load'
data['Load'] = data['Load'].fillna(0)
data.loc[data['Load']==0,'gen/load']='gen'

data['power']=data['quantity']+data['Load']
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0
...,...,...,...,...,...,...,...,...,...
1645660,2022-07-18 05:00:00,2022-07-18 05:30:00,UK,MAW,0,0.0,631.0,load,631.0
1645661,2022-07-18 05:30:00,2022-07-18 06:00:00,UK,MAW,0,0.0,696.0,load,696.0
1645662,2022-07-18 06:00:00,2022-07-18 06:30:00,UK,MAW,0,0.0,770.0,load,770.0
1645663,2022-07-18 06:30:00,2022-07-18 07:00:00,UK,MAW,0,0.0,840.0,load,840.0


In [225]:
# Extract date and hour
data['Date'] = data['StartTime'].dt.date
data['Hour'] = data['StartTime'].dt.hour
data

,StartTime,EndTime,AreaID,UnitName,PsrType,quantity,Load,gen/load,power,Date,Hour
0,2021-12-31 23:45:00,2022-01-01 00:00:00,DE,MAW,B01,4333.0,0.0,gen,4333.0,2021-12-31,23
1,2022-01-01 00:00:00,2022-01-01 00:15:00,DE,MAW,B01,4325.0,0.0,gen,4325.0,2022-01-01,0
2,2022-01-01 00:15:00,2022-01-01 00:30:00,DE,MAW,B01,4319.0,0.0,gen,4319.0,2022-01-01,0
3,2022-01-01 00:30:00,2022-01-01 00:45:00,DE,MAW,B01,4323.0,0.0,gen,4323.0,2022-01-01,0
4,2022-01-01 00:45:00,2022-01-01 01:00:00,DE,MAW,B01,4328.0,0.0,gen,4328.0,2022-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...
1645660,2022-07-18 05:00:00,2022-07-18 05:30:00,UK,MAW,0,0.0,631.0,load,631.0,2022-07-18,5
1645661,2022-07-18 05:30:00,2022-07-18 06:00:00,UK,MAW,0,0.0,696.0,load,696.0,2022-07-18,5
1645662,2022-07-18 06:00:00,2022-07-18 06:30:00,UK,MAW,0,0.0,770.0,load,770.0,2022-07-18,6
1645663,2022-07-18 06:30:00,2022-07-18 07:00:00,UK,MAW,0,0.0,840.0,load,840.0,2022-07-18,6


In [226]:
aggregated_data = data.groupby(['AreaID', 'gen/load', 'Date', 'Hour'])['power'].sum().reset_index()

aggregated_data['concatenated'] = aggregated_data['AreaID']  + aggregated_data['gen/load']

In [227]:
pivot = aggregated_data.pivot_table(
index=['Date', 'Hour'],
columns=['concatenated'],
values='power',
aggfunc='sum'
)
pivot

concatenated        DEgen    DEload   DKgen  DKload   HUgen   HUload   ITgen  \
Date       Hour                                                                
2021-12-31 23     39401.0   42196.0  3652.0  3314.0   285.0   4254.0  5801.0   
2022-01-01 0     157373.0  165125.0  3605.0  3218.0  1376.0  16457.0  5745.0   
           1     151747.0  160415.0  3309.0  3126.0  1526.0  15426.0  6228.0   
           2     147938.0  158035.0  3043.0  3080.0  1560.0  14781.0  6407.0   
           3     141816.0  157016.0  2852.0  3044.0  1287.0  14630.0  5760.0   
...                   ...       ...     ...     ...     ...      ...     ...   
2023-01-01 19    133596.0  177225.0  4168.0  3507.0   667.0  17873.0  3873.0   
           20    131681.0  172484.0  3975.0  3354.0   648.0  17141.0  3690.0   
           21    122205.0  166627.0  3498.0  3229.0   658.0  16259.0  3460.0   
           22    120116.0  153785.0  2879.0  3055.0   623.0  15273.0  3283.0   
           23     92697.0  110140.0     NaN     NaN   468.0  10900.0     NaN   

concatenated      ITload    NEgen   NEload   POgen   POload    SEgen   SEload  \
Date       Hour                                                                 
2021-12-31 23    21121.0   3554.0  10326.0  4641.0  14438.0  11060.0  15331.0   
2022-01-01 0     19756.0  12768.0  40706.0  4491.0  13935.0  11107.0  15331.0   
           1     18685.0  11131.0  39465.0  4436.0  13579.0  11036.0  15270.0   
           2     18124.0  11324.0  38923.0  4568.0  13397.0  10509.0  15150.0   
           3     18400.0  11290.0  38211.0  4559.0  13364.0  10770.0  15387.0   
...                  ...      ...      ...     ...      ...      ...      ...   
2023-01-01 19    24617.0   8730.0  47635.0  4968.0  15468.0  12347.0  16145.0   
           20    22944.0   7654.0  45689.0  4220.0  14698.0  11986.0  15869.0   
           21    20791.0   6577.0  43276.0  3525.0  13749.0  11227.0  15143.0   
           22    19198.0   7759.0  41269.0  3236.0  13061.0  10515.0  14697.0   
           23        NaN   5441.0  29948.0     NaN      NaN      NaN      NaN   

concatenated       SPgen   SPload  UKgen  UKload  
Date       Hour                                   
2021-12-31 23    11244.0  20827.0    NaN   677.0  
2022-01-01 0     10827.0  19530.0    NaN  1244.0  
           1     11140.0  18383.0    NaN  1131.0  
           2     11361.0  17680.0    NaN  1091.0  
           3     11247.0  17396.0    NaN   969.0  
...                  ...      ...    ...     ...  
2023-01-01 19    76316.0  98224.0  106.0     NaN  
           20    76276.0  97620.0   82.0     NaN  
           21    72280.0  91896.0   65.0     NaN  
           22    65968.0  84324.0   75.0     NaN  
           23    49836.0  59140.0   61.0     NaN  

[8785 rows x 18 columns]

In [228]:
import numpy as np
# Replace zeros with NaN
pivot.replace(0, np.nan, inplace=True)

# Drop rows where all elements are NaN
pivot=pivot.dropna()



In [229]:
pivot=pivot.reset_index()
pivot['Date'] = pd.to_datetime(pivot['Date'])
pivot = pivot[pivot['Date'].dt.year == 2022]
pivot

concatenated,Date,Hour,DEgen,DEload,DKgen,DKload,HUgen,HUload,ITgen,ITload,NEgen,NEload,POgen,POload,SEgen,SEload,SPgen,SPload,UKgen,UKload
0,2022-01-26,23,157769.0,229000.0,4695.0,3912.0,898.0,21808.0,5348.0,27465.0,15798.0,46721.0,5670.0,18504.0,11696.0,16174.0,7517.0,25797.0,865.0,1649.0
1,2022-01-27,0,164895.0,225922.0,4637.0,3825.0,913.0,20986.0,5162.0,26588.0,16103.0,45289.0,5599.0,18105.0,11209.0,15759.0,7110.0,24596.0,2042.0,1424.0
2,2022-01-27,1,171274.0,224274.0,4789.0,3786.0,840.0,20356.0,5582.0,26343.0,16450.0,44319.0,5659.0,18228.0,11231.0,15546.0,6449.0,24061.0,1852.0,1304.0
3,2022-01-27,2,176025.0,224262.0,4752.0,3838.0,855.0,20250.0,5295.0,26740.0,16663.0,43847.0,5648.0,18518.0,11636.0,15856.0,6469.0,24068.0,1344.0,1305.0
4,2022-01-27,3,174827.0,231010.0,4832.0,4065.0,895.0,20953.0,5634.0,28908.0,16825.0,44446.0,5659.0,19274.0,12357.0,16391.0,6424.0,24733.0,851.0,1282.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,2022-04-03,18,109615.0,222381.0,4239.0,4071.0,867.0,22140.0,5791.0,30605.0,3842.0,50416.0,3679.0,19404.0,13077.0,16916.0,19753.0,29804.0,2242.0,1862.0
1567,2022-04-03,19,110879.0,219818.0,4431.0,3808.0,877.0,21097.0,4977.0,28246.0,4734.0,49185.0,3913.0,18527.0,12471.0,16092.0,19212.0,28210.0,2340.0,1835.0
1568,2022-04-03,20,116236.0,215400.0,4340.0,3671.0,835.0,20082.0,4443.0,26001.0,7258.0,46835.0,4429.0,17458.0,12083.0,15467.0,18886.0,25811.0,2614.0,1635.0
1569,2022-04-03,21,119996.0,202810.0,4465.0,3484.0,794.0,19097.0,3489.0,23445.0,9527.0,44071.0,4572.0,16595.0,11771.0,15081.0,17887.0,23783.0,2667.0,1427.0


In [230]:
pivot.dropna()

concatenated,Date,Hour,DEgen,DEload,DKgen,DKload,HUgen,HUload,ITgen,ITload,NEgen,NEload,POgen,POload,SEgen,SEload,SPgen,SPload,UKgen,UKload
0,2022-01-26,23,157769.0,229000.0,4695.0,3912.0,898.0,21808.0,5348.0,27465.0,15798.0,46721.0,5670.0,18504.0,11696.0,16174.0,7517.0,25797.0,865.0,1649.0
1,2022-01-27,0,164895.0,225922.0,4637.0,3825.0,913.0,20986.0,5162.0,26588.0,16103.0,45289.0,5599.0,18105.0,11209.0,15759.0,7110.0,24596.0,2042.0,1424.0
2,2022-01-27,1,171274.0,224274.0,4789.0,3786.0,840.0,20356.0,5582.0,26343.0,16450.0,44319.0,5659.0,18228.0,11231.0,15546.0,6449.0,24061.0,1852.0,1304.0
3,2022-01-27,2,176025.0,224262.0,4752.0,3838.0,855.0,20250.0,5295.0,26740.0,16663.0,43847.0,5648.0,18518.0,11636.0,15856.0,6469.0,24068.0,1344.0,1305.0
4,2022-01-27,3,174827.0,231010.0,4832.0,4065.0,895.0,20953.0,5634.0,28908.0,16825.0,44446.0,5659.0,19274.0,12357.0,16391.0,6424.0,24733.0,851.0,1282.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,2022-04-03,18,109615.0,222381.0,4239.0,4071.0,867.0,22140.0,5791.0,30605.0,3842.0,50416.0,3679.0,19404.0,13077.0,16916.0,19753.0,29804.0,2242.0,1862.0
1567,2022-04-03,19,110879.0,219818.0,4431.0,3808.0,877.0,21097.0,4977.0,28246.0,4734.0,49185.0,3913.0,18527.0,12471.0,16092.0,19212.0,28210.0,2340.0,1835.0
1568,2022-04-03,20,116236.0,215400.0,4340.0,3671.0,835.0,20082.0,4443.0,26001.0,7258.0,46835.0,4429.0,17458.0,12083.0,15467.0,18886.0,25811.0,2614.0,1635.0
1569,2022-04-03,21,119996.0,202810.0,4465.0,3484.0,794.0,19097.0,3489.0,23445.0,9527.0,44071.0,4572.0,16595.0,11771.0,15081.0,17887.0,23783.0,2667.0,1427.0


In [231]:
for country in ['HU','IT','PO','SP','DE','DK','SE','NE']:
    pivot[country+'_surplus']=pivot[country+'gen']-pivot[country+'load']


In [232]:
import pandas as pd

# Assuming 'Date' is in datetime format, if not, convert it
pivot['Date'] = pd.to_datetime(pivot['Date'])

# Define European dates for seasons
spring_start = pd.to_datetime('2023-03-21')
summer_start = pd.to_datetime('2023-06-21')
autumn_start = pd.to_datetime('2023-09-22')
winter_start = pd.to_datetime('2023-12-21')

# Explicitly define bin edges and labels
bin_edges = [pd.Timestamp.min, spring_start, summer_start, autumn_start, winter_start, pd.Timestamp.max]
labels = ['min_to_spring', 'spring', 'summer', 'autumn', 'winter']

# Create a new column 'season' based on the defined seasons
pivot['season'] = pd.cut(
    pivot['Date'],
    bins=bin_edges,
    labels=labels,
    right=False
)

pivot['season'] = pivot['season'].replace('min_to_spring', 'winter')

# One-hot encode the 'season' column
pivot = pd.get_dummies(pivot, columns=['season'], drop_first=True)

# Extract day of the week and create 'weekend' column
pivot['day_of_week'] = pivot['Date'].dt.dayofweek
pivot['is_weekend'] = pd.get_dummies(pivot['day_of_week'].isin([5, 6]).astype(int), drop_first=True)

In [233]:
pivot.to_csv('../data/final_data.csv',index=False)